### Hinweis: Lösungs-Notebook

Dies ist das offizielle Lösungs- / Referenz-Notebook für den Qiskit Workshop.

- Teilnehmer sollten zuerst das Aufgaben-Notebook bearbeiten, bevor sie hier nachsehen.
- Die Inhalte zeigen Beispiel-Lösungen, saubere Schaltungen und exemplarische Auswertungen.
- Du kannst einzelne Zellen vergleichen oder selektiv ausführen.



Viel Erfolg & happy quantum hacking! ⚛️

>In den Lösungen ist der vollständige Code enthalten. Wir haben für die Demo den gesamten Code ausgelagert, damit die Bearbeitung einfacher wird.

# Qiskit Workshop Demo 

In [ ]:
# Abhängigkeiten sind bereits über requirements.txt installiert
# Falls du lokal ohne Installation arbeitest, entferne das Kommentarzeichen vor der nächsten Zeile:
# %pip install qiskit qiskit-aer matplotlib pylatexenc
import qiskit
import qiskit_aer
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram  # Für Histogramme weiter unten
import math  # Für Winkel / π etc.

# Gemeinsamer Simulator (kann in späteren Zellen direkt genutzt werden)
backend = Aer.get_backend('qasm_simulator')

print("Qiskit importiert (Basis-Imports + Simulator bereit)")

## Kapitel 1: Einfacher Einzel-Qubit-Circuit – Wie oft messen wir 0 oder 1? Lösung
Wir erstellen einen Circuit mit 1 Qubit und 1 klassischem Bit. Ein Hadamard-Gatter `H` bringt das Qubit in eine Superposition, sodass bei vielen Messungen ungefähr gleich oft `0` und `1` auftreten. Wir simulieren direkt und schauen uns die Häufigkeiten an.

Schaltungslösung:

![Kapitel 1 Schaltung](images/kapitel1_schaltung.png)

In [ ]:
# Einfaches Beispiel: 1 Qubit, Hadamard, Messung
# Ziel: Zeigen, dass ein einzelnes Qubit nach einem Hadamard ungefähr zu 50% als 0 und 50% als 1 gemessen wird.

# Anzahl der Wiederholungen von dem Circuit
shots = 100

# Erzeuge einen Circuit mit 1 Qubit und 1 klassischem Bit. Das klassische Bit wird benutzt, um das Ergebnis zu speichern
qc_simple = QuantumCircuit(1,1) # Qubit,Bit

# Hadamard-Gatter auf Qubit 0: |0> -> (|0> + |1>)/√2 (Superposition)
qc_simple.h(0)

# Messe Qubit 0 in das klassische Bit 0 (kollabiert Superposition zu 0 oder 1)
qc_simple.measure(0,0)

# Zeichne die Schaltung (mit 'mpl')
try:
    display(qc_simple.draw('mpl'))
except Exception:
    print(qc_simple.draw())

# Simuliere die Schaltung: 'qasm_simulator' führt viele Messwiederholungen (shots) aus
result = backend.run(qc_simple, shots=shots).result()  # 100 Wiederholungen -> statistische Verteilung

# Hole die Zählungen der gemessenen Bitstrings (hier nur '0' oder '1')
counts_simple = result.get_counts(qc_simple)
print("Häufigkeiten (Anzahl der Messungen pro Ergebnis):", counts_simple)

# In Wahrscheinlichkeiten / Prozent umrechnen
probs_simple = {k: v/shots for k,v in counts_simple.items()}
percent_line = " , ".join(f"{bit} = {p*100:.1f}%" for bit, p in sorted(probs_simple.items(), reverse=True))
print("Prozent:", percent_line)

from matplotlib.ticker import PercentFormatter
fig_or_ax = plot_histogram(probs_simple, title=f"Hadamard Ergebnis (Prozent) | {percent_line}")

# Achse auf Prozent formatieren
if hasattr(fig_or_ax, 'axes'):
    ax = fig_or_ax.axes[0]
else:
    ax = fig_or_ax
ax.set_ylim(0,1.0)
ax.set_ylabel('Prozent')
ax.yaxis.set_major_formatter(PercentFormatter(xmax=1, decimals=0))

# Anzeigen
try:
    from IPython.display import display as _display
    if hasattr(fig_or_ax, 'axes'):
        _display(fig_or_ax)
    else:
        _display(ax.figure)
except Exception:
    pass

## Kapitel 2: Wie erzwinge oder beeinflusse ich Messergebnisse?


### Variante 1: Immer 1 erzwingen mit X-Gate – Lösung
Wir setzen das Qubit per X-Gatter direkt in den Zustand |1>, damit die Messung (fast) immer 1 liefert.

Schaltungslösung:

![Kapitel 2 Schaltung](images/kpaitel2_schaltung.png)

In [ ]:
# Variante 1: Immer 1 messen (X)
# Ziel: Zeigen, wie wir das Ergebnis fest auf 1 setzen.

shots = 1000  # Anzahl der Wiederholungen

qc_one = QuantumCircuit(1,1)
qc_one.x(0)          # Macht aus |0⟩ ein |1⟩
qc_one.measure(0,0)  # Messergebnis ins klassische Bit

# Zeichne die Schaltung (mit 'mpl')
try:
    display(qc_one.draw('mpl'))
except Exception:
    print(qc_one.draw())

counts_one = backend.run(qc_one, shots=shots).result().get_counts(qc_one)
print("Variante 1 - Immer 1 (Counts):", counts_one)

# In Prozent/Wahrscheinlichkeiten umrechnen
probs_one = {k: v/shots for k,v in counts_one.items()}
# Prozent-String (sortiere nach Key absteigend, damit '1' zuerst erscheint)
percent_line = " , ".join(f"{bit} = {p*100:.1f}%" for bit, p in sorted(probs_one.items(), reverse=True))
print("Prozent:", percent_line)

from matplotlib.ticker import PercentFormatter
fig_or_ax = plot_histogram(probs_one, title=f"Variante 1: X -> immer 1 | {percent_line}")

# Falls eine Figure zurückkommt, Achse extrahieren
if hasattr(fig_or_ax, 'axes'):
    ax = fig_or_ax.axes[0]
else:
    ax = fig_or_ax
ax.set_ylim(0, 1.0)
ax.set_ylabel('Prozent')
ax.yaxis.set_major_formatter(PercentFormatter(xmax=1, decimals=0))

# Explizit anzeigen (Figure wurde einer Variable zugewiesen)
try:
    from IPython.display import display as _display
    if hasattr(fig_or_ax, 'axes'):
        _display(fig_or_ax)
    else:
        _display(ax.figure)
except Exception:
    pass

# (Optional) Altes Counts-Histogramm:
# plot_histogram(counts_one, title='Variante 1: X -> immer 1 (Counts)')

### Variante 2: Wahrscheinlichkeiten fein einstellen mit RY(θ)
Mit einer Rotation um die Y-Achse stellen wir kontinuierlich ein, wie oft 0 oder 1 fällt.


Schaltungslösung:

![Kapitel 2 Schaltung (Version 2)](images/kapitel2_schaltungv2.png)

In [ ]:
# Variante 2: Wahrscheinlichkeiten steuern mit RY(θ)
# Ziel: θ bestimmt, wie oft 1 herauskommt (glatte Einstellmöglichkeit)

shots = 500
θ = 60 * math.pi / 180    # 60 Grad direkt als Winkel, Umrechnung in Radiant

qc_bias = QuantumCircuit(1,1)
qc_bias.ry(θ,0)        # Drehung erzeugt Mischung zwischen 0 und 1
qc_bias.measure(0,0)

# Zeichne die Schaltung (mit 'mpl')
try:
    display(qc_bias.draw('mpl'))
except Exception:
    print(qc_bias.draw())

counts_bias = backend.run(qc_bias, shots=shots).result().get_counts(qc_bias)
print(f"Variante 2 - RY({θ:.2f}):", counts_bias)

# In Prozent / Wahrscheinlichkeiten
probs_bias = {k: v/shots for k,v in counts_bias.items()}
percent_line = " , ".join(f"{bit} = {p*100:.1f}%" for bit, p in sorted(probs_bias.items(), reverse=True))
print("Prozent:", percent_line)

from matplotlib.ticker import PercentFormatter
fig_or_ax = plot_histogram(probs_bias, title=f"Variante 2: RY({θ:.2f}) | {percent_line}")

# Figure/Axes handhaben wie zuvor
if hasattr(fig_or_ax, 'axes'):
    ax = fig_or_ax.axes[0]
else:
    ax = fig_or_ax
ax.set_ylim(0,1.0)
ax.set_ylabel('Prozent')
ax.yaxis.set_major_formatter(PercentFormatter(xmax=1, decimals=0))

try:
    from IPython.display import display as _display
    if hasattr(fig_or_ax, 'axes'):
        _display(fig_or_ax)
    else:
        _display(ax.figure)
except Exception:
    pass

## Kapitel 3: CNOT (CX) einfach erklärt (2 QUBITS)


### Wahrheitstabelle – Lösung
| Control | Target (vorher) | Target (nachher) | Ausgabe (Control Target) | Was passiert? |
|---------|-----------------|------------------|---------------------------|---------------|
| 0       | 0               | 0                | 00                        | Nichts        |
| 0       | 1               | 1                | 01                        | Nichts        |
| 1       | 0               | 1                | 11                        | Flip          |
| 1       | 1               | 0                | 10                        | Flip          |


### Kapitel 4: Mini Quantum 1-Bit-Addition – Lösung

In [ ]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram

# Eingaben:
a = 1
b = 1
    
qc = QuantumCircuit(3, 2)   # q0=a, q1=b, q2=Carry
qc.x(0)
qc.x(1)
qc.ccx(0,1,2)      # Carry = a AND b
qc.cx(0,1)         # Sum = a XOR b
qc.measure(1,0)    # Sum
qc.measure(2,1)    # Carry

backend = AerSimulator()
counts = backend.run(transpile(qc, backend), shots=512).result().get_counts()
display(qc.draw('mpl'))
plot_histogram(counts)


## ZUSATZ GUI Mini Quanten-"Taschenrechner": 1-Bit + 1-Bit
# Fortgeschritten: Erstelle das Quantum Circuit

Wir bauen eine ganz einfache Addition: Zwei klassische Bits (0 oder 1) werden addiert. Das Ergebnis besteht aus:
- SUM (Ergebnisstelle)
- CARRY (Übertrag, nur 1, falls 1+1 passiert)

Klassische Wahrheitstabelle:
```
a b | SUM CARRY
0 0 |  0    0
0 1 |  1    0
1 0 |  1    0
1 1 |  0    1   (weil 1+1 = 2 = (10)_2)
```

### Idee mit Qubits
Wir verwenden 3 Qubits:
- q0 = a (Eingabe)
- q1 = b (Eingabe / wird später zur SUM)
- q2 = 0 (Start bei |0⟩, speichert CARRY)

Schritte:
1. Falls a=1 oder b=1: Setze q0 oder q1 mit `x`.
2. `ccx(q0,q1,q2)` (Toffoli) schreibt a AND b in q2 → das ist unser CARRY.
3. `cx(q0,q1)` macht aus q1 jetzt a XOR b → das ist unsere SUM.
4. Messen: q1 = SUM, q2 = CARRY.

Fertig – das ist schon ein 1-Bit-Addierer.

### Merken
- CNOT = XOR (wenn du ein Ziel-Bit toggeln willst, falls Control=1)
- Toffoli = UND (speichert nur 1, wenn beide Controls 1 waren)


In [ ]:
# Interaktiver Half-Adder (Buttons)
from ipywidgets import ToggleButtons, IntSlider, Button, HBox, VBox, Output
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram

backend = Aer.get_backend('qasm_simulator')

# Widgets
a_in = ToggleButtons(options=[0,1], description='a:')
b_in = ToggleButtons(options=[0,1], description='b:')
shots_slider = IntSlider(value=256, min=64, max=4096, step=64, description='Shots:')
run_btn = Button(description='Addieren', button_style='success')
reset_btn = Button(description='Reset', button_style='warning')

out = Output()


def build_half_adder(a:int, b:int):
    qc = QuantumCircuit(3,2)
    if a: qc.x(0)
    if b: qc.x(1)
    qc.ccx(0,1,2)  # CARRY
    qc.cx(0,1)     # SUM in q1
    qc.measure(1,0)  # SUM -> c0
    qc.measure(2,1)  # CARRY -> c1
    return qc


def run(_):
    with out:
        out.clear_output(wait=True)
        a = a_in.value; b = b_in.value; shots = shots_slider.value
        qc = build_half_adder(a,b)
        try:
            display(qc.draw('mpl'))
        except Exception:
            print(qc.draw())
        counts = backend.run(qc, shots=shots).result().get_counts(qc)
        # Dominantes Ergebnis extrahieren
        dominant = max(counts.items(), key=lambda kv: kv[1])[0]
        # Bitstring = 'sc' (s=SUM in cbit0 rechts, c=CARRY in cbit1 links)
        sum_bit = dominant[1]
        carry_bit = dominant[0]
        value = int(carry_bit)*2 + int(sum_bit)  # Dezimalwert (0,1,2)
        print(f"Eingabe: a={a} b={b}")
        print(f"Bits: CARRY={carry_bit} SUM={sum_bit}  -> Ergebnis (dezimal) {a} + {b} = {value}")
        print(f"Binär (CARRY SUM) = {carry_bit}{sum_bit}  (entspricht der Zahl {value})")
        # Prozentanzeige vorbereiten
        total = sum(counts.values()) or 1
        probs = {k: v/total for k,v in counts.items()}
        perc = {k: f"{(v*100):5.1f}%" for k,v in probs.items()}
        print("Wahrscheinlichkeiten (%):", {k: perc[k] for k in sorted(probs.keys(), reverse=True)})
        from matplotlib.ticker import PercentFormatter
        fig_or_ax = plot_histogram(probs, title=f'Half-Adder a={a}+b={b} -> {value} (Prozent)')
        # Prozent-Achse formatieren
        if hasattr(fig_or_ax, 'axes'):
            ax = fig_or_ax.axes[0]
        else:
            ax = fig_or_ax
        ax.set_ylim(0,1.0)
        ax.set_ylabel('Prozent')
        ax.yaxis.set_major_formatter(PercentFormatter(xmax=1, decimals=0))
        try:
            from IPython.display import display as _display
            if hasattr(fig_or_ax, 'axes'):
                _display(fig_or_ax)
            else:
                _display(ax.figure)
        except Exception:
            pass


def reset(_):
    with out:
        out.clear_output()

run_btn.on_click(run)
reset_btn.on_click(reset)

ui = VBox([
    HBox([a_in, b_in, shots_slider]),
    HBox([run_btn, reset_btn]),
    out
])

display(ui)

***Deutsch-Algorithmus – Lösung*** 

>Kann bis zu 2 Minuten dauern, bis ein Ergebnis angezeigt wird.

In [ ]:
# Deutsch-Algorithmus – vollständige Referenzlösung
# Du kannst diesen Code studieren, verändern oder schrittweise selbst nachbauen.

from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import random

# ---------------------------------------------------------
# Orakel-Definitionen
# ---------------------------------------------------------
# Ziel: Orakel U_f wirkt wie |x>|y> -> |x>|y ⊕ f(x)>
# Wir brauchen je eine konstante und eine balancierte Variante.

def oracle_constant(value: int = 0) -> QuantumCircuit:
    """Konstantes Orakel.
    value = 0  => f(0)=f(1)=0  (macht nichts)
    value = 1  => f(0)=f(1)=1  (immer Flip des Ziel-Qubits)
    Rückgabe: 2-Qubit-QuantumCircuit, der das Orakel implementiert.
    """
    if value not in (0,1):
        raise ValueError("value muss 0 oder 1 sein")
    qc = QuantumCircuit(2, name=f"const_{value}")
    if value == 1:
        # Immer flippe das Ziel (Qubit 1) unabhängig von x
        qc.x(1)
    return qc

def oracle_balanced(kind: str = 'identity') -> QuantumCircuit:
    """Balanciertes Orakel.
    kind = 'identity' => f(x)=x
    kind = 'not'      => f(x)=¬x (also 1-x)
    Umsetzung über Phasen-/X-Gatter + CNOT.
    """
    if kind not in ('identity','not'):
        raise ValueError("kind muss 'identity' oder 'not' sein")
    qc = QuantumCircuit(2, name=f"bal_{kind}")
    if kind == 'identity':
        # f(x)=x -> klassisch: y ^= x => CNOT Kontrolle=0 -> Ziel=1
        qc.cx(0,1)
    else:  # kind == 'not'
        # f(x)=¬x -> y ^= (¬x) kann realisiert werden durch: X auf x, dann CNOT, dann X zurück
        qc.x(0)
        qc.cx(0,1)
        qc.x(0)
    return qc

# ---------------------------------------------------------
# Deutsch-Algorithmus Ausführung
# ---------------------------------------------------------
# Schema:
# 1. Initial: |0⟩|1⟩
# 2. H auf beide -> (|0⟩+|1⟩)/√2  ⊗ (|0⟩-|1⟩)/√2
# 3. Orakel U_f (kodiert f als Phase auf dem ersten Qubit)
# 4. H auf das erste Qubit
# 5. Messe das erste Qubit: 0 => konstant, 1 => balanciert

def run_deutsch(oracle: QuantumCircuit, shots: int = 1024):
    backend = Aer.get_backend('qasm_simulator')
    qc = QuantumCircuit(2,1)

    # Schritt 1: |0⟩|1⟩
    qc.x(1)  # zweites Qubit in |1⟩

    # Schritt 2: Hadamards
    qc.h(0)
    qc.h(1)

    # Schritt 3: Orakel einfügen (als Gate, um Name/Label zu behalten)
    qc.append(oracle.to_gate(), [0,1])

    # Schritt 4: Hadamard auf das erste Qubit
    qc.h(0)

    # Schritt 5: Messe nur das erste Qubit
    qc.measure(0,0)

    compiled = transpile(qc, backend)
    result = backend.run(compiled, shots=shots).result()
    counts = result.get_counts()

    # Dominantes Ergebnis auswerten
    dominant = max(counts.items(), key=lambda kv: kv[1])[0]
    classification = 'konstant' if dominant == '0' else 'balanciert'
    return qc, counts, classification

# ---------------------------------------------------------
# Demonstration aller vier Varianten
# ---------------------------------------------------------
const0 = oracle_constant(0)
const1 = oracle_constant(1)
bal_id = oracle_balanced('identity')
bal_not = oracle_balanced('not')

fälle = [
    ("konstant f(x)=0", const0),
    ("konstant f(x)=1", const1),
    ("balanciert f(x)=x", bal_id),
    ("balanciert f(x)=¬x", bal_not)
]

print("Deutsch-Algorithmus – Test aller Orakel (je 1 Ausführung mit vielen Shots):\n")
for label, oracle in fälle:
    qc_demo, counts, cls = run_deutsch(oracle, shots=512)
    print(f"{label:22s} -> Counts {counts} => erkannt: {cls}")
    try:
        display(qc_demo.draw('mpl'))
    except Exception:
        print(qc_demo.draw())

# ---------------------------------------------------------
# Zufallstest: Der Algorithmus soll korrekt klassifizieren.
# ---------------------------------------------------------
print("\nZufallstest:")
label_hidden, oracle_hidden = random.choice(fälle)
qc_rand, counts_rand, cls_rand = run_deutsch(oracle_hidden, shots=256)
print(f"Verstecktes Orakel: {label_hidden} -> Algorithmus sagt: {cls_rand}")
try:
    display(qc_rand.draw('mpl'))
except Exception:
    print(qc_rand.draw())
plot_histogram(counts_rand)

# Kurzer Hinweis:
print("\nInterpretation: Mess-0 => f war konstant, Mess-1 => f war balanciert. Deterministisch für ideale Orakel.")
